<a href="https://colab.research.google.com/github/SonnyK33/test/blob/master/Corporate_Credit_Class_11_25_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import sqlite3
import numpy as np
import pandas as pd
from sqlite3 import Error
import math 

#to-dos: 
#add prod growth, realized prices for commodities, fixed costs, capex/rev, cash, WC/rev, tax/rev to db


# conn = sqlite3.connect("test.db")
# c = conn.cursor()

# c.execute("Create TABLE stocks2 (date text)")

OilPrices = [40, 50, 60, 70]
NGPrices = [2.5,2.9,3.0,3.0]
currentYear = 2020.25
currentQuarter = 2


class CorporateCredit():
  def __init__(self, name, cusip, ticker, rating, sector, country):
    self.name = name
    self.cusip = cusip
    self.ticker = ticker
    self.rating = rating
    self.sector = sector
    self.country = country

  def Print_Description(self):
    print(f"Name: ",self.name)
    print(f"Ticker: ",self.ticker)
    print(f"Rating: ",self.rating)
    print(f"Sector: ",self.sector)
    print(f"Country: ",self.country)

  #def get historical levels

  
  #def settargets
    # sets target prices, yields, spreads
  
  def makeDBTable (self):
    Col = " ("
    DBname = input("Enter name of DB: ")
    Tablename = input("Enter name of table you want to make: ")

    while (True):
      n = 0      
      x = input("Enter column name ('q' when finished): ")
      if (x=='q'):
        break
      else:
        Col += (x+" text"+",")tc

      n+=1
    
    Col = Col[:-1]
    Col+=")"
    print(Col)

    conn = sqlite3.connect(DBname)
    c = conn.cursor()
    query = 'CREATE TABLE {}'.format(Tablename) + Col
    print(query)
    c.execute(query)
    
    #c.execute("CREATE TABLE {} ?",(Tablename,),Col)
    conn.commit()


    

  def makeProj (self, qtrs):
    
    FCF = ['Revenue','Variable Costs','Fixed Costs','EBITDA', 'Less: Capex', 'Less: Working Cap', 'Less: Taxes', 'FCF', 'Beginning Cash', 'Ending Cash']

    revArray = np.array([self.CurrentFCF['rev']*(1+self.CurrentFCF['rev_g'])**i for i in range(qtrs+1)],dtype=int)
    vcostsArray = np.array([self.CurrentFCF['var_costs']*(1+self.CurrentFCF['rev_g'])**i for i in range(qtrs+1)],dtype=int)
    fcostsArray = np.full(shape=qtrs+1,fill_value=self.CurrentFCF['f_costs'],dtype=int)
    ebitdaArray = revArray+vcostsArray+fcostsArray
    capexArray = revArray * self.CurrentFCF['capex/rev']
    wcArray = revArray * self.CurrentFCF['WC/rev']
    taxArray = ebitdaArray * self.CurrentFCF['tax']  #to fix --> need interest exp
    fcfArray = ebitdaArray + capexArray + wcArray + taxArray # need interest expense

    begCashArray = np.zeros(qtrs+1,dtype=int)
    begCashArray[0] = self.CurrentFCF['cash']
    endCashArray = np.zeros(qtrs+1,dtype=int)
    endCashArray[0] = fcfArray[0]+begCashArray[0]
      

    for i in range(1,qtrs+1):
      begCashArray[i] = endCashArray[i-1]        
      endCashArray[i] = begCashArray[i] + fcfArray[i]
        
      
    FCF_statement=[revArray,vcostsArray,fcostsArray,ebitdaArray,capexArray,wcArray,taxArray,fcfArray,begCashArray,endCashArray]
    FCF_df=pd.DataFrame(FCF_statement,columns = self.makeDateColumn(qtrs), index = FCF, dtype=int)

      # print(ebitdaArray)
      # print(begCashArray)
      # print(endCashArray)
      

    return FCF_df
      
  def makeDateColumn(self,qtrs):
    yearColumns = ['Q'+str(currentQuarter)+'/'+str(math.floor(currentYear))]
    for x in range (qtrs):
      yearColumns.append('Q'+str(((currentQuarter+x) % 4)+1)+'/'+str(math.floor(currentYear+(x+1)*0.25)))
    return yearColumns



class EnergyCredit(CorporateCredit):
  def __init__(self, name, cusip, ticker, rating, sector, country):      
    super().__init__(name, cusip, ticker, rating, sector, country)

     

 


class EPCredit(EnergyCredit):
  def __init__(self, name, cusip, ticker, rating, sector, country):
    super().__init__(name, cusip, ticker, rating, sector, country)
     # think about what parameters fully define an individual E&P Credit - Cash cost, production for each commodity, realized px, etc 

  def DB_Input(self):
    DebtEBITDA = float(input("Enter Debt/EBITDA: "))
    CashCost = float(input("Enter cash cost: "))
    OilProd = float(input("Enter Daily Oil Prod (bbd/d): "))
    NGProd = float(input("Enter Daily Natural Gas Prod (mmcf/d):  "))
    NGLProd = float(input("Enter Daily NGL Prod (bbd/d):  "))

    conn = sqlite3.connect("cc_v1.db")
    c = conn.cursor()
    c.execute("INSERT INTO EandP VALUES (?,?,?,?,?,?)",(self.name, CashCost, OilProd, NGLProd, NGProd, DebtEBITDA))
    conn.commit()
    # c.execute("SELECT * FROM EandP")
    # print(c.fetchone())



  def SetHedges(self):
    print(OilPrices[0])
    n = int(input("How many years are hedged: "))
    Hedge = [[0 for i in range(n)] for j in range(7)]
    #[year 0, year 1, ...]
    #[oil volume]
    #[oil realized price]
    #[NGL volume]
    #[NGL realized price]
    #[NG volume]
    #[NG realized price]

    #print(type(n))
    for i in range(0,n):
      print("Year "+str(i+1)+" :")
      TypeHedge = input("Oil hedged through (f)utures or (o)ptions?")

      if TypeHedge=='f':
        Hedge[1][i] = int(input("Oil volume hedged: "))
        Hedge[2][i] = int(input("Hedged Price: "))
      
      elif TypeHedge=='o':
        Hedge[1,i] = int(input("Oil volume hedged: "))
        Floor = int(input("Put Strike: "))
        Ceiling = int(input("Written Call Strike: "))
        SoldPut = int(input("Written Put Strike: "))
        Hedge[2][i] = OilPrices[i] - max(OilPrices[i]-Ceiling,0)+max(Floor-OilPrices[i],0)-max(SoldPut-OilPrices[i],0)

      TypeHedge = input("Gas hedged through (f)utures or (o)ptions?")

      if TypeHedge=='f':
        Hedge[5][i] = int(input("Gas volume hedged: "))
        Hedge[6][i] = int(input("Hedged Price: "))
      
      elif TypeHedge=='o':
        Hedge[5][i] = int(input("Gas volume hedged: "))
        Floor = int(input("Put Strike: "))
        Ceiling = int(input("Written Call Strike: "))
        SoldPut = int(input("Written Put Strike: "))
        Hedge[6][i] = OilPrices[i] - max(OilPrices[i]-Ceiling,0)+max(Floor-OilPrices[i],0)-max(SoldPut-OilPrices[i],0)

      TypeHedge = input("NGL hedged through (f)utures or (o)ptions?")

      if TypeHedge=='f':
        Hedge[3][i] = int(input("NGL volume hedged: "))
        Hedge[4][i] = int(input("Hedged Price: "))
      
      elif TypeHedge=='o':
        Hedge[3][i] = int(input("NGL volume hedged: "))
        Floor = int(input("Put Strike: "))
        Ceiling = int(input("Written Call Strike: "))
        SoldPut = int(input("Written Put Strike: "))
        Hedge[4][i] = OilPrices[n] - max(OilPrices[i]-Ceiling,0)+max(Floor-OilPrices[i],0)-max(SoldPut-OilPrices[i],0)
      
    return(Hedge)
      
  def setCurrentFCF(self):
    conn = sqlite3.connect("cc_v1_test.db")
    c = conn.cursor()
    c.execute("SELECT OilperDay from EandP WHERE name=?",(self.name,))
    #print(c.fetchone()[0])
    #c.fetchone()[0]
    #print(c.fetchone()[0])
    OilProd = int(c.fetchone()[0])
    c.execute("SELECT NGLperDay from EandP WHERE name=?",(self.name,))
    NGLProd = int(c.fetchone()[0])
    c.execute("SELECT NGperDay from EandP WHERE name=?",(self.name,))
    NGProd = int(c.fetchone()[0])
    c.execute("SELECT CashCost from EandP WHERE name=?",(self.name,))
    CashCost = int(c.fetchone()[0])

    TotalProd = OilProd + NGLProd + NGProd/6
    Oil_rev = OilProd * OilPrices[0] * 0.365/4
    NGL_rev = NGLProd * OilPrices[0] * 0.365/4
    NG_rev = NGProd * NGPrices[0] * 0.365/4

    Rev = Oil_rev + NGL_rev + NG_rev
    Prod_g = 0.01
    Rev_g = Prod_g
    Var_costs = -(CashCost * TotalProd * 0.365/4)
    F_costs = -70
    Cash = 162
    Capex_rev = -0.27
    WC_rev = -0.06
    Taxes = -0.11 #% of EBITDA    
    
    self.CurrentFCF = {'rev': Rev, 'rev_g': Rev_g, 'var_costs': Var_costs, 'f_costs': F_costs, 'cash': Cash, 'capex/rev': Capex_rev, 'WC/rev': WC_rev, 'tax':Taxes}

    









A = EPCredit("Apache","xx","APA",["BBB-","BBB"],"Energy","USA")

A.Print_Description()
# #A.DB_Input()
# A.setCurrentFCF()
# print(A.CurrentFCF)
# fcf = A.makeProj(4)
# fcf
#Hedge = A.SetHedges()
#print(Hedge)
A.makeDBTable()




Name:  Apache
Ticker:  APA
Rating:  ['BBB-', 'BBB']
Sector:  Energy
Country:  USA
Enter name of DB: test.db
Enter name of table you want to make: able
Enter column name ('q' when finished): a
Enter column name ('q' when finished): b
Enter column name ('q' when finished): c
Enter column name ('q' when finished): q
 (a text,b text,c text)
CREATE TABLE able (a text,b text,c text)
